TODO:



*  Modularize data generation
    * Structure data-gen class
*  Expand dataset selection
    * Enron dataset
    * Discord scraper
    * Stanford paper (add link to this)



In [ ]:
# INSTALL GENERAL DEPENDENCIES

#! python3
!pip install openai
# Downgrade pandas to be compatible with Colab
!pip install pandas==1.1.0

#libraries
import os
import openai

import datetime



In [ ]:
# Connect environment variables. If there is not a vars.env file in your drive, colab-env will create one. You can directly add variables
# to the file in a text editor, or use the following sample code to add. (Remember to delete the calls before committing, or it defeats the purpose!)
!pip install colab-env --upgrade
import colab_env
#colab_env.envvar_handler.add_env('var','value', overwrite=True)




In [ ]:
!more gdrive/My\ Drive/vars.env

In [ ]:
# CONNECT KAGGLE

from google.colab import drive
drive.mount('/content/drive')

!pip install -q kaggle
!pip install -q kaggle-cli
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/
!cat ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

# Format of kaggle download command (non-competition datasets). ENRON EMAILS DATASET CRASHES JESSIE'S LAPTOP
#!kaggle datasets download -d wcukierski/enron-email-dataset -p enron_emails

# SAMPLE CODE; DESIRED DATASET TOO BIG

# Alternate way to download kaggle files
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
# Download all
api.dataset_download_files('imdevskp/corona-virus-report')

# Download 1
api.dataset_download_file('imdevskp/corona-virus-report','covid_19_clean_complete.csv')

zf = ZipFile('covid_19_clean_complete.csv.zip')
#extracted data is saved in the same directory as notebook
zf.extractall() 
zf.close()

import pandas as pd
data=pd.read_csv('covid_19_clean_complete.csv')


In [ ]:
# CONNECT TWITTER 

#! python3

#libraries
import pandas as pd
import tweepy
import csv
import os
import datetime

#Twitter API credentials
consumer_key = 'key'
consumer_secret = 'secret'
access_key = 'key'
access_secret = 'secret'


API_KEY = "api_key"



def validate_twitter(consumer_key, consumer_secret, access_key, access_secret):
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_key, access_secret)

  api = tweepy.API(auth)

  return api

def sort_tweets_by_time_chunk(input_df, column_name='Date', time_chunk="week"):
    for x in range(52):
      df = input_df[input_df[column_name].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{screen_name}.csv')

# get an arbitrary # of tweets
def get_tweets(screen_name, num, api):
    new_tweets = api.user_timeline(screen_name=screen_name, count=num)
    
    return new_tweets


def get_all_tweets(screen_name, api):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name,
                                       count=200,
                                       max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print(f"...{len(alltweets)} tweets downloaded so far")

    #transform the tweepy tweets into a 2D array that will populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text]
                 for tweet in alltweets]
                 
    # Transposed in the repl.it
    tweets_df = pd.DataFrame(outtweets,
                             columns=["Tweet_id", "Date", "Tweet_Text"])
    
    # TODO: incorporate this start to error handling
    # except BaseException as e:
    #       print('failed on_status', str(e))
    #       time.sleep(3)

  # For each week in the year, create dir if ! exists, filter df, write to csv, and save to folder
  # See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekofyear.html  
    '''
    for x in range(52):
      df = tweets_df[tweets_df['Date'].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{screen_name}.csv')
    '''
    return tweets_df

    
# If using datetime object, can use this function to get the week for an individual tweet
# Note that every python data type handles this slightly differently! Check the type!
def make_chunks(data, tuple_position=1):
  # 0 = year, 1 = week, 2 = day
  weeks = []
  for x in data:
    chunk = x.created_at.isocalendar()[tuple_position]
    weeks.append(chunk)
  return weeks

# #tweets
# tweets = pd.read_csv('bored_tweets.csv')

# #remove links
# no_links = tweets[~tweets.text.str.contains("http")]
# no_links.head()

# # only tweets
# just_tweets = no_links.text

# # make everything a string
# text = str(just_tweets)


api = validate_twitter(consumer_key, consumer_secret, access_key, access_secret)
test = get_all_tweets('BoredElonMusk', api)
print(test)


getting tweets before 1476076237432901632
...400 tweets downloaded so far
getting tweets before 1472332160543559681
...600 tweets downloaded so far
getting tweets before 1468350290092908544
...800 tweets downloaded so far
getting tweets before 1461187944123748357
...1000 tweets downloaded so far
getting tweets before 1456288787831537665
...1200 tweets downloaded so far
getting tweets before 1453461088251506692
...1400 tweets downloaded so far
getting tweets before 1445819076073328644
...1600 tweets downloaded so far
getting tweets before 1442996049262493697
...1800 tweets downloaded so far
getting tweets before 1439633102150635522
...2000 tweets downloaded so far
getting tweets before 1436130768128917531
...2200 tweets downloaded so far
getting tweets before 1433763634119729154
...2400 tweets downloaded so far
getting tweets before 1430462321915088896
...2600 tweets downloaded so far
getting tweets before 1427079018990374914
...2800 tweets downloaded so far
getting tweets before 142084

In [ ]:
# CONNECT DISCORD #

import requests

print(os.environ["DISCORD_BOT_TOKEN"])

headers = {"Authorization": f"Bot {bot_token}"}

test_channel_id = 885948250253848588
test_guild_id = 647140436686667776

channel_path = f'https://discord.com/api/channels/{test_channel_id}/messages'

guild_path_channels = f'https://discord.com/api/guilds/{test_guild_id}/channels'


res = requests.get(channel_path, headers=headers).json()

for x in res:
  print(x)



NameError: ignored

In [ ]:
# SUPPORT FUNCTIONS #
import sys
import shutil

# Lazy util function
def check_type(data):
  print(type(data))

def NUKE_ALL_FOLDERS_AND_FILES():
  print("Nuclear launch detected")
  # Get directory name
  # TODO: Update this to be more flexible, but it works for now
  for x in range(1,52):
    try:
        shutil.rmtree(f'week-{x}')
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

#NUKE_ALL_FOLDERS_AND_FILES()

#os.listdir()

In [ ]:


# TODO: Objectify functions

def generate_df(input_df, file_path_name):
  '''
  - For x in df, grab x['date'] and split into day/month/year/week of year
  - Make columns for each val and assign accordingly
  - This will result in one large spreadsheet that you can filter
  '''
  pass

def parse_by_weeks(input_df, file_path_name):
    for x in range(52):
      df = input_df[input_df['Date'].dt.isocalendar().week == x]
      print(df.count)
      # Only make folders and files for weeks where there were tweets
      if not df.empty:
        if not os.path.exists(f'week-{x}'):
          os.makedirs(f'week-{x}')
        df.to_csv(f'week-{x}/{file_path_name}.csv')
    return input_df

  
generate_data(test, "test")

In [ ]:
### DRAFT Collection Object
import requests
import pandas as pd

class DataCollection():
  def __init__(self, token, output_columns=["id", "post_date", "author", "content"]):
        self.token = token
        self.output_columns = output_columns
  def make_df(self, input_arr):
    df = pd.DataFrame(self, input_arr, columns=self.output_columns)
    return df
  def help(self, service=None):
    message = "This will be a spot to outline the various functions and required bits for each service. Pass arg 'service=' for help with specific services"
    # Match is not yet implemented in Colab. Sad day.
    '''
    match service:
            case ‘discord’ : message += "Discord requires a BOT token"
    '''
    if service == 'discord':
      message += '\n Discord requires a BOT token'
    return message

# TOOD: redo pagination
class TwitterCollection(DataCollection):
  # NOTES: Twitter's requests are formatted such that you need to look up the user id first.
  def get_user(self, screen_name):
    headers = {"Authorization": f"Bearer {self.token}"}
    req = requests.get(f'https://api.twitter.com/2/users/by/username/{screen_name}', headers=headers)
    return req
    
  def get_tweets(self, screen_name, num):
    user_id = self.get_user(screen_name).json()['data']['id']
    #tweets_list = (screen_name=screen_name, count=num)
    #req = requests.get( )
    headers = {"Authorization": f"Bearer {self.token}"}
    req = requests.get(f'https://api.twitter.com/2/users/{user_id}/tweets', headers=headers) 
    return req


class DiscordCollection(DataCollection):
  def get_channel_messages(self, channel, before=None, after=None, around=None, count=None):
    headers = {"Authorization": f"Bot {self.token}"}
    count = count if count else 100
    before = f'&before={before}' if before else ""
    after = f'&after={after}' if after else ""
    around = f'&around={around}' if around else ""
    channel_path = f'https://discord.com/api/channels/{channel}/messages?&limit={count}'
    allmessages = []
    try:
        new_messages = requests.get(channel_path, headers=headers).json() 
        allmessages.extend(new_messages)
        print(len(allmessages))
        #save the id of the oldest tweet less one
        oldest = int(allmessages[-1]['id']) - 1
        print(int(len(allmessages)) < count)
        #keep grabbing messages until there are no messages left to grab
        if int(len(allmessages)) < count:
          while len(new_messages) > 0:
              print(len(new_messages))
              print(f"getting messages before {oldest}")
              channel_path = f'https://discord.com/api/channels/{channel}/messages?limit={count}&before={oldest}'
              new_messages = requests.get(channel_path, headers=headers).json() 

              allmessages.extend(new_messages)

              print(f"...{len(allmessages)} messages downloaded so far")

              try:
                  oldest = int(allmessages[-1]['id']) - 1
              except:
                  print("Done")
                  break
          print(len(allmessages))
        return allmessages
    except Exception as e: print(f'Error: {e}')

  def get_user_messages(self, user, before=None, after=None, around=None, count=None):
    # Should return messages from a specific user
    pass

  def make_arr(self, messages):
      message_arr = [{m['id'], m['timestamp'], m['author']['username'], m['content']} for m in messages]
      return message_arr


class KaggleCollection(DataCollection):
  pass


# Reddit requires a temporary oAuth token.
# NOTE THAT YOU MUST UPDATE YOUR USER AGENT STRING WITH YOUR OWN INFO IF NOT USING MY CREDS OR YOUR APP WILL BE BANNED
def get_reddit_token(client, secret, username, password, user_agent_string='data_test:0.0.1 ((by /u/person_of_note)'):
  # note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
    auth = requests.auth.HTTPBasicAuth(client, secret)

    # here we pass our login method (password), username, and password
    data = {'grant_type': 'password',
            'username': username,
            'password': password}

    # setup our header info, which gives reddit a brief description of our app
    headers = {'User-Agent': user_agent_string}

    # send our request for an OAuth token
    res = requests.post('https://www.reddit.com/api/v1/access_token',
                        auth=auth, data=data, headers=headers)

    # convert response to JSON and pull access_token value
    oauth_token = res.json()['access_token']
    # add authorization to our headers dictionary
    return oauth_token

# TODO: check for oauth expiry and get new token automatically
class RedditCollection(DataCollection):
  def get_subreddit(self, subreddit, filter="Hot"):
    headers = {'Authorization': f"bearer {self.token}"}
    # while the token is valid (~2 hours) we just add headers=headers to our requests
    res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{filter}",
                      headers=headers)
    print(res.status_code)
    if res.status_code == 200:
      for post in res.json()['data']['children']:
          print(post['data']['title'])
    elif res.status_code > 400:
      print('Too many requests, please try again later')
    else:
      print("Invalid token, please get a new token using the get_token function")

class SlackCollection(DataCollection):
  pass


In [ ]:
testDiscord = DiscordCollection(os.environ["DISCORD_BOT_TOKEN"])
discord_messages = testDiscord.get_channel_messages(885948250253848588, count=2)
discord_arr = testDiscord.make_arr(discord_messages)
discord_df = testDiscord.make_df(discord_arr)
print(discord_df)

2
False


ModuleNotFoundError: ignored

OBJECT DESIGN NOTES:

Possibilities:

* Object takes in a df that has been formatted elsewhere. Object would focus primarily on methods that manipulate the data into different file formats and sorting methods.
    * Relies on the user to clean the data and make dataframes that conform to a predtermined column format, which is less magical than the next design
    * Follows DRY, SRP, KISS and YAGNI
    * Pseudo-code examples:
        * `weeks_dataset = new DataSet.weeks(df, filter_by="weeks")`
        * `weeks_and_years_dataset = new DataSet.weeks(df, filter_by=['weeks','years'])`

* Include methods for scraping the most common datasets, something like:
    * Object would attempt to parse out a date object by searching for a column/split point marked as "date," "created_at" or similar
    * Relies less on end user
    * Runs the risk of getting really bloated and trying too hard to cover every edge case for every random data source
    * pseudo-code examples:
        * `kaggle_data = new DataSet.kaggle(credentials, dataset_name)` (are Kaggle datasets standardized, or are there a ton of different presentations? I think the latter)
        * `discord_data = new DataSet.discord(credentials, args* kwargs*)`
        * `twittter_data = new DataSet.twitter(< adapt existing function from this notebook>)`

# TYPE ONE:
# This is not exact; it is late at night and mostly an example, I don't remember the exact syntax off the top of my head
class CleanDataSet:
    def __init__(self, df, file_prefix):
        self.input_df = input_df
        self.file_prefix = file_prefix
        
    def parse_by_weeks(self):
      for x in range(52):
        df = input_df[input_df['Date'].dt.isocalendar().week == x]
        print(df.count)
        # Only make folders and files for weeks where there were tweets
        if not df.empty:
          if not os.path.exists(f'week-{x}'):
            os.makedirs(f'week-{x}')
          df.to_csv(f'week-{x}/{file_prefix}.csv')
      return input_df



# TYPE TWO:

class ScrapeDataSet:
    def __init__(self, data_source, file_prefix):
        self.data_source = data_source
        self.file_prefix = file_prefix
        
    def kaggle(credentials, dataset):
      # Fetch dataset by path (see cell )

In [ ]:
#HELPER FUNCTIONS FOR ENRON DATA PARSING

import email

email_df = pd.read_csv('enron_path') #just change this to the right path and run these two cells

def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [ ]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

In [ ]:
#get topics using empath

#!pip install empath

from empath import Empath
lexicon = Empath()

#sentence to analyze
content = emails_df['content'][2]

#analyze
nlp = lexicon.analyze(content, normalize=True)

#print the content
print(content)
#print topics with scores greater than 0.5 
for key, v in nlp.items():
    if v > 0.05:
        print(key, v)


In [ ]:
#join empath results with emails_df where empath results are greater than 0.5
#subset emails df to make this run faster for testing
emails_df_subset = emails_df[:100]

emails_df_subset['empath'] = ""
for i, row in emails_df_subset.iterrows():
    content = row['content']
    nlp = lexicon.analyze(content, normalize=True)
    for key, v in nlp.items():
        if v > 0.05:
            emails_df_subset.at[i, 'empath'] = key

emails_df_subset['empath'].head()

In [ ]:
#train gpt3 classifier on emapth topcis
import openai
import config
#for gpt2 if needed
#import gpt_2_simple as gpt2



openai.api_key = config.openai_key

#create new df for training
train_df = emails_df_subset[['content', 'empath']]
#make sure no null values
train_df = train_df[train_df['empath'] != ""]
#im getting errors about the length of the content examples so I need to only select ones that are less than 500 characters till i figure out how to fix this
train_df = train_df[train_df['content'].str.len() < 30]

#create a dictionary of examples
examples_dict = dict(zip(train_df.content, train_df.empath))
#for the sake of this test we will reduce the dict down to the first 3 items or gpt3 will error -- will need to train on a csv for more
examples_dict = {k: examples_dict[k] for k in list(examples_dict)[:3]}
#print(examples_dict)

#turn examples into a list of lists for gpt3 formatting
examples = list(map(list, examples_dict.items()))

#get unique values from the list for the training labels
labels = list(set([x for x in examples_dict.values()]))
#print(labels)

#select random content from emails_df to classify
from random import randrange
num = randrange(200000)

query = emails_df['content'][num]

#remove URLS from query
import re
query = re.sub(r'http\S+', '', query)

#print(query)

def gpt_classify(examples, labels, query):
  response = openai.Classification.create(
        search_model="ada",
        model="curie",
        examples=[examples],
        query=query,
        labels=labels,
      )
  return response.label

test = gpt_classify(examples, labels, query)

print(test)